In [1]:
import numpy as np
import netCDF4
import pandas as pd
import xarray as xr
from eofs.xarray import Eof
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import RandomizedSearchCV
from esem import rf_model
from esem.utils import leave_one_out
import joblib

# data_path = './data/train_val/'
data_path = "/glade/u/home/okyang/input_data/"

min_co2 = 0.
max_co2 = 9500
def normalize_co2(data):
    return data / max_co2

def un_normalize_co2(data):
    return data * max_co2

min_ch4 = 0.
max_ch4 = 0.8
def normalize_ch4(data):
    return data / max_ch4

def un_normalize_ch4(data):
    return data * max_ch4


def create_predictor_data(data_sets, n_eofs=5):
    """
    Args:
        data_sets list(str): names of datasets
        n_eofs (int): number of eofs to create for aerosol variables
    """
        
    # Create training and testing arrays
    if isinstance(data_sets, str):
        data_sets = [data_sets]
    X = xr.concat([xr.open_dataset(data_path + f"inputs_{file}.nc") for file in data_sets], dim='time')
    X = X.assign_coords(time=np.arange(len(X.time)))

    # Compute EOFs for BC
    bc_solver = Eof(X['BC'])
    bc_eofs = bc_solver.eofsAsCorrelation(neofs=n_eofs)
    bc_pcs = bc_solver.pcs(npcs=n_eofs, pcscaling=1)

    # Compute EOFs for SO2
    so2_solver = Eof(X['SO2'])
    so2_eofs = so2_solver.eofsAsCorrelation(neofs=n_eofs)
    so2_pcs = so2_solver.pcs(npcs=n_eofs, pcscaling=1)

    # Convert to pandas
    bc_df = bc_pcs.to_dataframe().unstack('mode')
    bc_df.columns = [f"BC_{i}" for i in range(n_eofs)]

    so2_df = so2_pcs.to_dataframe().unstack('mode')
    so2_df.columns = [f"SO2_{i}" for i in range(n_eofs)]

    # Bring the emissions data back together again and normalise
    inputs = pd.DataFrame({
        "CO2": normalize_co2(X["CO2"].data),
        "CH4": normalize_ch4(X["CH4"].data)
    }, index=X["CO2"].coords['time'].data)

    # Combine with aerosol EOFs
    inputs = pd.concat([inputs, bc_df, so2_df], axis=1)
    return inputs, (so2_solver, bc_solver)


def get_test_data(file, eof_solvers, n_eofs=5):
    """
    Args:
        file str: name of datasets
        n_eofs (int): number of eofs to create for aerosol variables
        eof_solvers (Eof_so2, Eof_bc): Fitted Eof objects to use for projection
    """
        
    # Create training and testing arrays
    X = xr.open_dataset(data_path + f"inputs_{file}.nc")
        
    so2_pcs = eof_solvers[0].projectField(X["SO2"], neofs=5, eofscaling=1)
    so2_df = so2_pcs.to_dataframe().unstack('mode')
    so2_df.columns = [f"SO2_{i}" for i in range(n_eofs)]

    bc_pcs = eof_solvers[1].projectField(X["BC"], neofs=5, eofscaling=1)
    bc_df = bc_pcs.to_dataframe().unstack('mode')
    bc_df.columns = [f"BC_{i}" for i in range(n_eofs)]

    # Bring the emissions data back together again and normalise
    inputs = pd.DataFrame({
        "CO2": normalize_co2(X["CO2"].data),
        "CH4": normalize_ch4(X["CH4"].data)
    }, index=X["CO2"].coords['time'].data)

    # Combine with aerosol EOFs
    inputs = pd.concat([inputs, bc_df, so2_df], axis=1)
    return inputs


def create_predictdand_data(data_sets):
    if isinstance(data_sets, str):
        data_sets = [data_sets]
    Y = xr.concat([xr.open_dataset(data_path + f"outputs_{file}.nc") for file in data_sets], dim='time').mean("member")
    # Convert the precip values to mm/day
    Y["pr"] *= 86400
    Y["pr90"] *= 86400
    return Y


def get_rmse(truth, pred):
    weights = np.cos(np.deg2rad(truth.lat))
    return np.sqrt(((truth - pred)**2).weighted(weights).mean(['lat', 'lon'])).data

2025-11-24 18:33:58.740891: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-24 18:33:59.191355: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-24 18:34:08.726527: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
train_files = ["historical", "ssp126", "ssp370", 'hist-aer', 'hist-GHG']
tasFiles = ["historical", "ssp126", "ssp370", "hist-aer", "hist-GHG"]
# Create training and testing arrays
X, solvers = create_predictor_data(train_files)
Y = create_predictdand_data(tasFiles)

In [3]:
rfTas = RandomForestRegressor(random_state=0)
rfPr = RandomForestRegressor(random_state=0)
rfPr90 = RandomForestRegressor(random_state=0)
rfDTR = RandomForestRegressor(random_state=0)

In [4]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 300, num = 5)]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5,55, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [5, 10, 15, 25]
# Minimum number of samples required at each leaf node
min_samples_leaf = [4, 8, 12]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [5]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [6]:
rfTasRandom = RandomizedSearchCV(estimator = rfTas, param_distributions = random_grid, n_iter = 29, cv = 3, verbose=2, n_jobs = -1)
rfPrRandom = RandomizedSearchCV(estimator = rfPr, param_distributions = random_grid, n_iter = 29, cv = 3, verbose=2, n_jobs = -1)
rfPr90Random = RandomizedSearchCV(estimator = rfPr90, param_distributions = random_grid, n_iter = 29, cv = 3, verbose=2, n_jobs = -1)
rfDTRRandom = RandomizedSearchCV(estimator = rfDTR, param_distributions = random_grid, n_iter = 29, cv = 3, verbose=2, n_jobs = -1)

In [7]:
Y.compute()

<xarray.Dataset> Size: 184MB
Dimensions:                    (time: 667, lat: 96, lon: 144)
Coordinates:
  * lat                        (lat) float64 768B -90.0 -88.11 ... 88.11 90.0
  * lon                        (lon) float64 1kB 0.0 2.5 5.0 ... 355.0 357.5
  * time                       (time) int64 5kB 1850 1851 1852 ... 2013 2014
    quantile                   float64 8B 0.9
Data variables:
    diurnal_temperature_range  (time, lat, lon) float32 37MB 0.03264 ... -0.3354
    tas                        (time, lat, lon) float32 37MB 0.32 ... 2.611
    pr                         (time, lat, lon) float32 37MB 0.02337 ... 0.04197
    pr90                       (time, lat, lon) float64 74MB 0.02468 ... 0.08654

In [7]:
y_inp_tas  = Y["tas"].stack(space=("lat", "lon"))
y_inp_pr   = Y["pr"].stack(space=("lat", "lon"))
y_inp_pr90 = Y["pr90"].stack(space=("lat", "lon"))
y_inp_dtr  = Y["diurnal_temperature_range"].stack(space=("lat", "lon"))

In [8]:
rf_tas = rfTasRandom.fit(X, y_inp_tas)

Fitting 3 folds for each of 29 candidates, totalling 87 fits
[CV] END bootstrap=True, max_depth=55, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=250; total time=  51.2s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=15, n_estimators=100; total time=  30.4s


/glade/u/home/okyang/.local/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=True, max_depth=55, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=250; total time=  45.8s
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=12, min_samples_split=5, n_estimators=250; total time=  59.0s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=8, min_samples_split=25, n_estimators=100; total time=  14.3s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=8, min_samples_split=25, n_estimators=100; total time=  14.7s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=8, min_samples_split=25, n_estimators=100; total time=  16.0s
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=12, min_samples_split=5, n_estimators=250; total time= 1.0min
[CV] END bootstrap=True, max_depth=55, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=250; total time=  46.8s
[CV] END bootstrap=False, max_dep

In [9]:
joblib.dump(rf_tas, "model.pkl")

['model.pkl']

In [10]:
rf_pr = rfPrRandom.fit(X, y_inp_pr)

Fitting 3 folds for each of 29 candidates, totalling 87 fits
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=150; total time=  30.1s
[CV] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=15, n_estimators=300; total time=  40.6s


/glade/u/home/okyang/.local/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=150; total time=  32.2s
[CV] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=15, n_estimators=300; total time=  40.1s
[CV] END bootstrap=False, max_depth=35, max_features=sqrt, min_samples_leaf=12, min_samples_split=10, n_estimators=100; total time=  25.4s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=  49.2s
[CV] END bootstrap=False, max_depth=35, max_features=sqrt, min_samples_leaf=12, min_samples_split=5, n_estimators=200; total time=  46.1s
[CV] END bootstrap=False, max_depth=35, max_features=sqrt, min_samples_leaf=12, min_samples_split=10, n_estimators=100; total time=  23.1s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=  44.1s
[CV] END bootstrap=False, max_depth=3

In [11]:
joblib.dump(rf_pr, "modelpr.pkl")

['modelpr.pkl']

In [12]:
rf_pr90 = rfPr90Random.fit(X, y_inp_pr90)

Fitting 3 folds for each of 29 candidates, totalling 87 fits
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=12, min_samples_split=15, n_estimators=100; total time=  23.7s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=12, min_samples_split=15, n_estimators=100; total time=  23.8s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=12, min_samples_split=15, n_estimators=100; total time=  26.7s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=8, min_samples_split=10, n_estimators=200; total time=  54.9s


/glade/u/home/okyang/.local/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=250; total time= 1.6min
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=8, min_samples_split=5, n_estimators=150; total time=  40.3s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=250; total time= 1.4min
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=8, min_samples_split=10, n_estimators=200; total time= 1.0min
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=8, min_samples_split=5, n_estimators=150; total time=  44.3s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=12, min_samples_split=25, n_estimators=300; total time= 1.2min
[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=250; total time= 1.5min
[CV] END bootstrap=False, max_de

In [13]:
joblib.dump(rf_pr90, "modelpr90.pkl")

['modelpr90.pkl']

In [14]:
rf_dtr = rfDTRRandom.fit(X, y_inp_dtr)

Fitting 3 folds for each of 29 candidates, totalling 87 fits
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=4, min_samples_split=15, n_estimators=200; total time= 1.2min
[CV] END bootstrap=True, max_depth=35, max_features=sqrt, min_samples_leaf=12, min_samples_split=15, n_estimators=100; total time=  13.5s
[CV] END bootstrap=True, max_depth=35, max_features=sqrt, min_samples_leaf=12, min_samples_split=15, n_estimators=100; total time=  13.7s
[CV] END bootstrap=True, max_depth=35, max_features=sqrt, min_samples_leaf=12, min_samples_split=15, n_estimators=100; total time=  14.6s
[CV] END bootstrap=True, max_depth=35, max_features=sqrt, min_samples_leaf=8, min_samples_split=15, n_estimators=200; total time=  31.1s
[CV] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=8, min_samples_split=10, n_estimators=200; total time=  34.9s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=

/glade/u/home/okyang/.local/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [15]:
joblib.dump(rf_dtr, "modeldtr.pkl")

['modeldtr.pkl']

[CV] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=12, min_samples_split=5, n_estimators=300; total time=  39.0s
[CV] END bootstrap=False, max_depth=25, max_features=sqrt, min_samples_leaf=8, min_samples_split=5, n_estimators=150; total time=  40.0s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=8, min_samples_split=25, n_estimators=300; total time= 1.2min
[CV] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=25, n_estimators=100; total time=  13.2s
[CV] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=25, n_estimators=100; total time=  13.0s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=12, min_samples_split=5, n_estimators=150; total time=  35.8s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=8, min_samples_split=15, n_estimators=100; total time=  30.4s
[CV] END bootstrap=False, max_depth=45, m

In [16]:
X_test = get_test_data('ssp245', solvers)
Y_test = create_predictdand_data(['ssp245'])

[CV] END bootstrap=True, max_depth=35, max_features=sqrt, min_samples_leaf=8, min_samples_split=15, n_estimators=200; total time=  31.2s
[CV] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=8, min_samples_split=10, n_estimators=200; total time=  31.0s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=150; total time=  28.6s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=12, min_samples_split=10, n_estimators=250; total time=  50.6s
[CV] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=12, min_samples_split=25, n_estimators=250; total time=  37.0s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time= 1.1min
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=8, min_samples_split=25, n_estimators=200; total time=  46.2s
[CV] END bootstrap=False, max_depth=5, 

In [17]:
tas_truth = Y_test["tas"]
pr_truth = Y_test["pr"]
pr90_truth = Y_test["pr90"]
dtr_truth = Y_test["diurnal_temperature_range"]

In [19]:
m_out_t = rf_tas.predict(X_test)

In [21]:
m_out_p = rf_pr.predict(X_test)
m_out_p90 = rf_pr90.predict(X_test)
m_out_d = rf_dtr.predict(X_test)

In [26]:
m_out_tas = m_out_t.reshape(86, 96, 144)
m_out_pr = m_out_p.reshape(86, 96, 144)
m_out_pr90 = m_out_p90.reshape(86, 96, 144)
m_out_dtr = m_out_d.reshape(86, 96, 144)

In [27]:
print(f"RMSE: {get_rmse(tas_truth[35], m_out_tas[35])}")
print(f"RMSE: {get_rmse(tas_truth[85], m_out_tas[85])}")
print(f"RMSE: {get_rmse(tas_truth[30:40], m_out_tas[30:40])}")
print(f"RMSE: {get_rmse(tas_truth[75:], m_out_tas[75:])}")
print(f"RMSE: {get_rmse(tas_truth[35:], m_out_tas[35:])}")
print("\n")

print(f"RMSE: {get_rmse(dtr_truth[35], m_out_dtr[35])}")
print(f"RMSE: {get_rmse(dtr_truth[85], m_out_dtr[85])}")
print(f"RMSE: {get_rmse(dtr_truth[30:40], m_out_dtr[30:40])}")
print(f"RMSE: {get_rmse(dtr_truth[75:], m_out_dtr[75:])}")
print(f"RMSE: {get_rmse(dtr_truth[35:], m_out_dtr[35:])}")
print("\n")

print(f"RMSE: {get_rmse(pr_truth[35], m_out_pr[35])}")
print(f"RMSE: {get_rmse(pr_truth[85], m_out_pr[85])}")
print(f"RMSE: {get_rmse(pr_truth[30:40], m_out_pr[30:40])}")
print(f"RMSE: {get_rmse(pr_truth[75:], m_out_pr[75:])}")
print(f"RMSE: {get_rmse(pr_truth[35:], m_out_pr[35:])}")
print("\n")

print(f"RMSE: {get_rmse(pr90_truth[35], m_out_pr90[35])}")
print(f"RMSE: {get_rmse(pr90_truth[85], m_out_pr90[85])}")
print(f"RMSE: {get_rmse(pr90_truth[30:40], m_out_pr90[30:40])}")
print(f"RMSE: {get_rmse(pr90_truth[75:], m_out_pr90[75:])}")
print(f"RMSE: {get_rmse(pr90_truth[35:], m_out_pr90[35:])}")

RMSE: 0.3933021822828737
RMSE: 1.1830596314001198
RMSE: [0.40568224 0.39790269 0.46512724 0.46625477 0.43156453 0.39330218
 0.56938062 0.6807958  0.62321142 0.51160139]
RMSE: [1.25981729 1.2926717  1.12531066 1.11802042 1.19958431 1.17783401
 1.07573113 1.1128248  1.28374934 1.19935643 1.18305963]
RMSE: [0.39330218 0.56938062 0.6807958  0.62321142 0.51160139 0.47193171
 0.54437246 0.71965468 0.64657379 0.71002978 0.7973534  0.74567085
 0.7488372  0.81121037 0.83688292 0.82793188 0.91397732 1.05265191
 0.97172602 0.93468838 1.05432975 1.17001527 1.23199152 1.24678142
 1.28269405 1.3277437  1.23008823 1.10743215 1.13112484 1.12798254
 1.11720532 1.35894281 1.30787726 1.0939208  1.15175184 1.36768797
 1.31542083 1.05788825 1.03431996 1.10250896 1.25981729 1.2926717
 1.12531066 1.11802042 1.19958431 1.17783401 1.07573113 1.1128248
 1.28374934 1.19935643 1.18305963]


RMSE: 0.13556584887502277
RMSE: 0.18643362986685963
RMSE: [0.14038983 0.12830037 0.13390882 0.14206303 0.13740251 0.13556585